**Задание 1. Напишите функцию, которая принимает DataFrame и возвращает имена двух столбцов с наибольшей положительной корреляцией**

*Этот блокнот объясняет действие функции и ее составляющих (В основном это нужно для меня самой)*

Посмотрим на **датафрейм**, который мы будем обрабатывать:

In [69]:
import pandas as pd

data = {
    'Region': [2, 1, 3, 1],
    'Sales': [200, 150, 300, 250],
    'Advertising': [50, 40, 70, 60]
}

df = pd.DataFrame(data)
df

,Region,Sales,Advertising
0,2,200,50
1,1,150,40
2,3,300,70
3,1,250,60


Считаем **матрицу корреляций** Пирсона:

In [70]:
corr_mat = df.corr()
corr_mat

,Region,Sales,Advertising
Region,1.0000,0.6742,0.6742
Sales,0.6742,1.0000,1.0000
Advertising,0.6742,1.0000,1.0000


С помощью `.shape` можно получить измерения датафрейма (количество строк, столбцов и т.д, если измерений больше 2). 

In [72]:
corr_mat.shape

(3, 3)

Как можно заметить, матрица квадратная, симметричная и с единицами на диагонали. Мы будем брать из нее нижний треугольник для поиска максимальных положительных корреляций. 

*Пояснения:*
- `corr_mat.shape[0]` - **количество** строк/столбцов в матрице
  
- Корреляционная матрица является датафреймом, в котором есть **столбцы** с **названиями**: 'Region', 'Sales', 'Advertising' и **строки** с **индексами** (тоже **названиями**): 'Region', 'Sales', 'Advertising'. 

  При этом с помощью `.iloc` можно обращаться не к **названиям**, а к их **номерам**: 0, 1, 2. Это работает как для **строк**, так и для **столбцов**.

- `row` - **номер строки** матрицы (в контексте датафрейма - **номер индекса** строки)
- `col` - **номер столбца** матрицы
- `corr_mat.iloc[row, col]` - значение **корреляции** для строки `row` и столбца `col`

Вот так можно просто перебрать элементы в нижнем треугольнике:

In [ ]:
for row in range(corr_mat.shape[0]):
    for col in range(row):
        print(corr_mat.iloc[row, col])

0.674199862463242
0.6741998624632421
1.0


Запишем эти значения вместе с названиями строк и столбцов в отдельную переменную. Используем для этого генератор списков (list comprehension)

- В переменной `corrs` бедет лежать список кортежей по три элемента: значение корреляции, название строки и название столбца, для которых была высчитана эта корреляция. (`corr_mat.columns[row]` и `corr_mat.columns[col]` - **названия** строк и столбцов в датафрейме с **номерами** `row` и `col`)

- `row` и `col` перебираются в нижнем треугольнике матрицы

In [ ]:
corrs = [
    (corr_mat.iloc[row, col], corr_mat.columns[row], corr_mat.columns[col]) 
    for row in range(corr_mat.shape[0]) for col in range(row)  # нижний треугольник
]
corrs

[(np.float64(0.674199862463242), 'Sales', 'Region'),
 (np.float64(0.6741998624632421), 'Advertising', 'Region'),
 (np.float64(1.0), 'Advertising', 'Sales')]

Теперь находим из этого списка кортежей - кортеж с **максимальным** значением корреляции. 
- Параметр `key=` в функции `max()` отвечает за ключ, по которому мы ищем максимальное значение. В данном случае, мы ищем по первым элементам кортежа - значению корреляции, т.е. `x[0]` в лямбда-функции

Для получения только **названий строк**, сделаем срез кортежа `[1:]`

In [79]:
max_correlation = max(corrs, key=lambda x: x[0])
print(max_correlation)
print(max_correlation[1:])

(np.float64(1.0), 'Advertising', 'Sales')
('Advertising', 'Sales')


Итого получили функцию:

In [ ]:
def get_highest_correlation(df: pd.DataFrame) -> tuple:
    """Возвращает имена двух столбцов с наибольшей положительной корреляцией для датафрейма"""
    # Вычисляем корреляционную матрицу по методу Пирсона:
    corr_mat = df.corr()

    # Запишем все корреляции из нижнего треугольника матрицы (с названиями строк и столбцов)
    corrs = [
        (corr_mat.iloc[row, col], corr_mat.columns[row], corr_mat.columns[col]) 
        for row in range(corr_mat.shape[0]) for col in range(row)  # нижний треугольник
    ]

    # Найдем наибольший элемент из полученных корреляций и возвращаем только названия столбцов:
    return max(corrs, key=lambda x: x[0])[1:]

get_highest_correlation(df)

('Advertising', 'Sales')